# Working with parquet files

## Objective

+ In this assignment, we will use the data downloaded with the module `data_manager` to create features.

(11 pts total)

## Prerequisites

+ This notebook assumes that price data is available to you in the environment variable `PRICE_DATA`. If you have not done so, then execute the notebook `production_2_data_engineering.ipynb` to create this data set.


+ Load the environment variables using dotenv. (1 pt)

In [2]:
# Write your code below.
%load_ext dotenv
%dotenv 


In [3]:
import dask.dataframe as dd

+ Load the environment variable `PRICE_DATA`.
+ Use [glob](https://docs.python.org/3/library/glob.html) to find the path of all parquet files in the directory `PRICE_DATA`.

(1pt)

In [4]:
import os
from glob import glob

# Write your code below.

directory_path='../../05_src/data/prices'
PRICE_DATA = os.getenv("PRICE_DATA")


from glob import glob

parquet_files = glob(os.path.join(directory_path, "**/*.0.parquet"),recursive=True)
len(parquet_files)

11207

For each ticker and using Dask, do the following:

+ Add lags for variables Close and Adj_Close.
+ Add returns based on Adjusted Close:
    
    - `returns`: (Adj Close / Adj Close_lag) - 1

+ Add the following range: 

    - `hi_lo_range`: this is the day's High minus Low.

+ Assign the result to `dd_feat`.

(4 pt)

In [6]:
# Write your code below.
import dask.dataframe as dd
# I will read the parquet files and see the table below for a succesfull completion
dd_px = dd.read_parquet(parquet_files)

In [7]:
dd_px.head()

Price,Date,Adj Close,Close,High,Low,Open,Volume,sector,subsector,year
ticker,,,,,,,,,,
A,2000-01-03 00:00:00+00:00,43.463043,51.502148,56.464592,48.193848,56.330471,4674353,Health Care,Life Sciences Tools & Services,2000
A,2000-01-04 00:00:00+00:00,40.142929,47.567955,49.266811,46.316166,48.730328,4765083,Health Care,Life Sciences Tools & Services,2000
A,2000-01-05 00:00:00+00:00,37.652863,44.617310,47.567955,43.141991,47.389126,5758642,Health Care,Life Sciences Tools & Services,2000
A,2000-01-06 00:00:00+00:00,36.219193,42.918453,44.349072,41.577251,44.080830,2534434,Health Care,Life Sciences Tools & Services,2000
A,2000-01-07 00:00:00+00:00,39.237465,46.494991,47.165592,42.203148,42.247852,2819626,Health Care,Life Sciences Tools & Services,2000


In [9]:
#Let's check the table if everything work well
dd_px.head()

Price,Date,Adj Close,Close,High,Low,Open,Volume,sector,subsector,year
ticker,,,,,,,,,,
A,2000-01-03 00:00:00+00:00,43.463043,51.502148,56.464592,48.193848,56.330471,4674353,Health Care,Life Sciences Tools & Services,2000
A,2000-01-04 00:00:00+00:00,40.142929,47.567955,49.266811,46.316166,48.730328,4765083,Health Care,Life Sciences Tools & Services,2000
A,2000-01-05 00:00:00+00:00,37.652863,44.617310,47.567955,43.141991,47.389126,5758642,Health Care,Life Sciences Tools & Services,2000
A,2000-01-06 00:00:00+00:00,36.219193,42.918453,44.349072,41.577251,44.080830,2534434,Health Care,Life Sciences Tools & Services,2000
A,2000-01-07 00:00:00+00:00,39.237465,46.494991,47.165592,42.203148,42.247852,2819626,Health Care,Life Sciences Tools & Services,2000


In [10]:
#Step 1: Add lags variables Close and Adj Close.
dd_px=dd_px.groupby('ticker', group_keys=False).apply(lambda x: x.assign(Adj_Close_lag = x['Adj Close'].shift(1)))

C:\Users\Zekiye\AppData\Local\Temp\ipykernel_19760\3806333837.py:2: UserWarning: `meta` is not specified, inferred from partial data. Please provide `meta` if the result is unexpected.
  Before: .apply(func)
  After:  .apply(func, meta={'x': 'f8', 'y': 'f8'}) for dataframe result
  or:     .apply(func, meta=('x', 'f8'))            for series result
  dd_px=dd_px.groupby('ticker', group_keys=False).apply(lambda x: x.assign(Adj_Close_lag = x['Adj Close'].shift(1)))


In [11]:
# Step 2: Calculate returns based on Adjusted Close
dd_px['returns'] = (dd_px['Adj Close'] / dd_px['Adj_Close_lag']) - 1

# Step 3: Calculate the high-low range for each day
dd_px['hi_lo_range'] = dd_px['High'] - dd_px['Low']

# Assign the resulting DataFrame to `dd_feat`
dd_feat = dd_px[['Date', 'Close', 'Adj Close', 'Adj_Close_lag', 'returns', 'hi_lo_range']]

In [15]:
dd_feat.head()

Price,Date,Close,Adj Close,Adj_Close_lag,returns,hi_lo_range
ticker,,,,,,
HUM,2002-01-02 00:00:00+00:00,11.50,10.268301,NaN,NaN,0.559999
HUM,2002-01-03 00:00:00+00:00,11.50,10.268301,10.268301,0.000000,0.309999
HUM,2002-01-04 00:00:00+00:00,11.55,10.312946,10.268301,0.004348,0.290000
HUM,2002-01-07 00:00:00+00:00,11.48,10.250441,10.312946,-0.006061,0.260000
HUM,2002-01-08 00:00:00+00:00,11.46,10.232581,10.250441,-0.001742,0.280001


+ Convert the Dask data frame to a pandas data frame. 
+ Add a rolling average return calculation with a window of 10 days.
+ *Tip*: Consider using `.rolling(10).mean()`.

(3 pt)

In [14]:
# Write your code below.

import pandas as pd

# Step 4: Convert Dask DataFrame to pandas DataFrame
df_feat = dd_feat.compute()

# Step 5: Add a 10-day rolling average return calculation
df_feat_rtn = (df_feat
               .groupby(['ticker'])
               .apply(lambda x: x.assign(ma_return=x['returns']
                                         .rolling(window=10).mean())))

Please comment:

+ Was it necessary to convert to pandas to calculate the moving average return?
+ Would it have been better to do it in Dask? Why?

(1 pt)

In [ ]:
# It was not strictly necessary to convert to a pandas DataFrame to calculate the moving average return; 
# it could indeed have been calculated directly within Dask. Performing this calculation in Dask offers 
# several benefits, particularly for larger datasets, as Dask is designed for out-of-core computation 
# and can handle data that doesn’t fit into memory.

## Criteria

The [rubric](./assignment_1_rubric_clean.xlsx) contains the criteria for grading.

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-1`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_1.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.